In [ ]:
!pip install pycryptodome
!pip install matplotlib
!pip install tree-sitter

In [ ]:
import csv
import re
from ast import literal_eval
from itertools import combinations_with_replacement, permutations

from Crypto.Hash import keccak

In [ ]:
# generate a simplified database containing just the function signatures and
# their original hashes, and save them to disk

HASH_COL = 1
SIGNATURE_COL = 2

with open("./base.csv") as bf:
    br = csv.reader(bf)
    with open("./signature-hash.csv", mode="w") as sf:
        sw = csv.writer(sf, delimiter=';')
        for brow in br:
            sw.writerow([
                brow[SIGNATURE_COL].removeprefix('"').removesuffix('"'),
                brow[HASH_COL]
            ])

In [ ]:
# check for hashing inconsistencies and print out erroneous entries

with open("./signature-hash.csv") as sf:
    sr = csv.reader(sf, delimiter=';')

    nrow = -1
    for row in sr:
        nrow += 1
        if nrow == 0:
            continue

        # compute hash for the signature
        keccak_hash = keccak.new(digest_bits=256)
        keccak_hash.update(bytearray(row[0], encoding="utf-8"))
        digest = keccak_hash.hexdigest()

        # compare the computed hash with the one in the DB
        if row[1] != digest:
            print(nrow)
            print(row, digest)

In [ ]:
# XXX: this is the solution I came up with on Saturday to parse the
# solidity function signatures and retrieve the func name and parameter types.
# It considers any tuple of parameter types as a single type, and does not
# bother with recursion.
# An better solution is to write up a simple grammar
# and compile it with tree_sitter to generate a proper parser.
# That solution is provided below.

def split_signature(signature):
    # regular expression pattern to match function name and parameter types
    pattern = r'^(\w+)\((.*)\)$'
    match = re.match(pattern, signature)
    if match:
        function_name = match.group(1)
        parameter_types_str = match.group(2)
        parameter_types = []
        current_type = ''
        parentheses_count = 0
        
        # iterate through each character in the parameter types string
        for char in parameter_types_str:
            if char == ',' and parentheses_count == 0:
                parameter_types.append(current_type.strip())
                current_type = ''
            else:
                current_type += char
                if char == '(':
                    parentheses_count += 1
                elif char == ')':
                    parentheses_count -= 1
        
        # Append the last parameter type
        if current_type.strip():
            parameter_types.append(current_type.strip())
        
        return function_name, parameter_types
    else:
        return None, None

In [ ]:
# Example
split_signature("deposit(address,uint256,int128,(address,address,int128,int128,bool,bool,uint256)")

In [ ]:
# Sane solution using a simple grammar
# defined in tree-sitter-signatures/grammar.js
# the grammar does not include arrays of types for now

# This solution also allows for parsing of the entire
# database in one go, instead of parsing each line.
# But for the sake of consistency with the function above
# it is used in the same manner.

from tree_sitter import Language, Node, Parser

Language.build_library(
    'build/signatures.so',
    ['tree-sitter-signatures']
)

SIG_LANG = Language("build/signatures.so", "signatures")
parser = Parser()
parser.set_language(SIG_LANG)

query = SIG_LANG.query(
"""
(function_definition
    (function_name) @fname
    (parameters
        (parameter) @param
    ) @params
)
"""
)

def split_signature(signature):
    tree = parser.parse(
        bytes(signature, 'utf-8')
    )

    matches = query.matches(tree.root_node)
    if not matches:
        return None, None
        
    param_types = []
    for match in matches:
        param = match[1]["param"]

        # since the encoding is in utf-8, you can index
        # characters with their byte number. 
        param_types.append(signature[param.start_byte:param.end_byte])

    func_name = matches[0][1]["fname"]
    return signature[func_name.start_byte:func_name.end_byte], param_types

In [ ]:

source = """
myFunction(uint, uint, (string, bool))
"""

split_signature(source)

In [ ]:
fnames = set()
fparams = set()
nparams = []

# get all function names and parameters included in the DB
with open("./signature-hash.csv") as sf:
    sr = csv.reader(sf, delimiter=';')

    nrow = -1
    for row in sr:
        nrow += 1
        if nrow == 0:
            continue

        fn, ps = split_signature(row[0])

        fnames.add(fn)
        
        if not ps:
            continue
        for p in ps:
            fparams.add(p)

        nps = len(ps)
        nparams.append(nps)

In [ ]:
# This could of course be replaced by actually
# specifying the native types in tree-sitter-signatures/grammar.js
# and recompiling (with the command tree-sitter generate).
def is_native_type_or_tuple(typ):
    """
    Check if a given type is a native Solidity type or a tuple of native Solidity types.
    """
    native_types = [
        'address', 'bool', 'string', 'int', 'uint', 'byte', 'bytes',
        'bytes1', 'bytes2', 'bytes3', 'bytes4', 'bytes5', 'bytes6',
        'bytes7', 'bytes8', 'bytes9', 'bytes10', 'bytes11', 'bytes12',
        'bytes13', 'bytes14', 'bytes15', 'bytes16', 'bytes17', 'bytes18',
        'bytes19', 'bytes20', 'bytes21', 'bytes22', 'bytes23', 'bytes24',
        'bytes25', 'bytes26', 'bytes27', 'bytes28', 'bytes29', 'bytes30',
        'bytes31', 'bytes32', 'int8', 'int16', 'int24', 'int32', 'int40',
        'int48', 'int56', 'int64', 'int72', 'int80', 'int88', 'int96',
        'int104', 'int112', 'int120', 'int128', 'int136', 'int144',
        'int152', 'int160', 'int168', 'int176', 'int184', 'int192',
        'int200', 'int208', 'int216', 'int224', 'int232', 'int240',
        'int248', 'int256', 'uint8', 'uint16', 'uint24', 'uint32', 'uint40',
        'uint48', 'uint56', 'uint64', 'uint72', 'uint80', 'uint88', 'uint96',
        'uint104', 'uint112', 'uint120', 'uint128', 'uint136', 'uint144',
        'uint152', 'uint160', 'uint168', 'uint176', 'uint184', 'uint192',
        'uint200', 'uint208', 'uint216', 'uint224', 'uint232', 'uint240',
        'uint248', 'uint256'
    ]
    
    if typ.endswith(']'):
        typ, n = typ.split('[', 1)
        try:
            int(n)
        except ValueError:
            return False

    # TODO: check for invalid types within tuples
    if not typ.startswith('(') and typ not in native_types:
        return False

    return True

In [ ]:
fparams = {p for p in fparams if is_native_type_or_tuple(p)}

In [ ]:
import matplotlib.pyplot as plt

nparams = [n for n in nparams if n < 15]

# The point of this histogram is to estimate
# how many parameters in a function signature
# it is reasonable to asume as a maximum number
# of parameters.
plt.hist(nparams)

In [ ]:
# this is the number of computations
# that we would be performing
# if we no further filtered the data
len(fnames) * len(fparams) ** 4
# of course, it is unfeasible for now

In [ ]:
# We are also interested in checking for permutations
# within the words in the function name.
def split_camelcase(name):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', name)
    return [m.group(0) for m in matches]

def correct_camelcase(split_name):
    if len(split_name) == 1:
        return split_name[0].lower()
        
    camel_case = split_name[0].lower()
    for w in split_name[1:]:
        camel_case += w.capitalize()
    return camel_case

In [ ]:
# XXX: replace the actual lists with a subset, to be able to compute:
# even though we have the logic to include tuples as custom types, we
# are limiting ourselves to these basic ones for now
fparams = {"uint8", "uint256", "uint", "address", "bytes", "bool"}
fnames = list(fnames)[:20]

# The results will be written to disk in a file called results.csv
fnno = 0
with open("./results.csv", mode="w") as of:
    ow = csv.writer(of, delimiter=';')
    ow.writerow(["Signature", "Hash"])
    for func_name in fnames:
        fnno += 1
        print(fnno)
        # Iterate within every permutation of the camelCase name
        for name_perm in permutations(split_camelcase(func_name)):
            # go from 0 to 4 parameters
            for i in range(5):
                combs = combinations_with_replacement(fparams, i)
                # go through every combination of parameter types for that
                # amount of parameters
                for c in combs:
                    sig = correct_camelcase(name_perm) + "(" + ",".join(c) + ")"
                    keccak_hash = keccak.new(digest_bits=256)
                    keccak_hash.update(bytearray(sig, encoding="utf-8"))
                    digest = keccak_hash.hexdigest()
                    ow.writerow([sig, digest])